In [1]:
import tweepy
import pymysql
import json
from dateutil import parser
import time
import os
import subprocess

In [2]:
import mysql.connector

In [4]:
consumer_key = '*'
consumer_secret= '*'
access_token= '*-*'
access_token_secret = '*'
password = '*'

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(access_token, access_token_secret)

In [11]:
def connect(username, created_at, tweet, retweet_count, place , location):
	"""
	connect to MySQL database and insert twitter data
	"""
	try:
		con = mysql.connector.connect(host = 'localhost',
		database = 'twitterdb', user='root', password = password, charset = 'utf8')

		if con.is_connected():
			"""
			Insert twitter data
			"""
			cursor = con.cursor()
			# twitter, golf
			query = "INSERT INTO twitterdata (username, created_at, tweet, retweet_count, place, location) VALUES (%s, %s, %s, %s, %s, %s)"
			cursor.execute(query, (username, created_at, tweet, retweet_count, location, place))
			con.commit()
            
	except Error as e: 
		print(e)   
	cursor.close()
	con.close()

	return

In [12]:
# Tweepy class to access Twitter API
class Streamlistener(tweepy.StreamListener):
	

	def on_connect(self):
		print("You are connected to the Twitter API")


	def on_error(self):
		if status_code != 200:
			print("error found")
			# returning false disconnects the stream
			return False

	"""
	This method reads in tweet data as Json
	and extracts the data we want.
	"""
	def on_data(self,data):
		
		try:
			raw_data = json.loads(data)

			if 'text' in raw_data:
				 
				username = raw_data['user']['screen_name']
				created_at = parser.parse(raw_data['created_at'])
				tweet = raw_data['text']
				retweet_count = raw_data['retweet_count']

				if raw_data['place'] is not None:
					place = raw_data['place']['country']
					print(place)
				else:
					place = None
				

				location = raw_data['user']['location']

				#insert data just collected into MySQL database
				connect(username, created_at, tweet, retweet_count, place, location)
				print("Tweet colleted at: {} ".format(str(created_at)))
		except Error as e:
			print(e)

In [ ]:
if __name__== '__main__':

	# # #Allow user input
	# track = []
	# while True:

	# 	input1  = input("what do you want to collect tweets on?: ")
	# 	track.append(input1)

	# 	input2 = input("Do you wish to enter another word? y/n ")
	# 	if input2 == 'n' or input2 == 'N':
	# 		break
	
	# print("You want to search for {}".format(track))
	# print("Initialising Connection to Twitter API....")
	# time.sleep(2)

	# authentification so we can access twitter
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_token, access_token_secret)
	api =tweepy.API(auth, wait_on_rate_limit=True)

	# create instance of Streamlistener
	listener = Streamlistener(api = api)
	stream = tweepy.Stream(auth, listener = listener)

	track = ['data science', 'machine learning', 'Artificial intelligence', 'Deep Learning', 'ml','dl']
	#track = ['nba', 'cavs', 'celtics', 'basketball']
	# choose what we want to filter by
	stream.filter(track = track, languages = ['en'])

In [13]:
import pandas as pd

In [29]:
twitterdb = pymysql.connect("localhost", "root", "ck1234", "twitterdb")
query = "select * from twitterdata"
data = pd.read_sql(query, twitterdb)

In [32]:
data.shape

(2205, 7)

In [48]:
cursor = twitterdb.cursor()
query = "select created_at,username, tweet from twitterdb.twitterdata"
#cursor.execute(query)
#data = cursor.fetchall()
# store in dataframe
df = pd.read_sql(query,twitterdb,columns=['time','username','tweet'])
df.head()

,created_at,username,tweet
0,2020-05-30 19:41:10,joker_fantasy_m,RT @Jav_grandpa: [DASD-669] An Erotically Brai...
1,2020-05-30 19:41:10,CyberSecurityN8,RT @gp_pulipaka: Best #Books That Will Teach Y...
2,2020-05-30 19:41:12,sectest9,RT @gp_pulipaka: Best #Books That Will Teach Y...
3,2020-05-30 19:41:13,ishancarter,RT @CaptMarkKelly: I’m running for Senate to b...
4,2020-05-30 19:41:13,TGNewsfall,#Microsoft is reducing dozens of MSN records p...


In [60]:
cursor.close()

In [42]:
# -*- coding: utf-8 -*-
import mysql.connector 
from mysql.connector import Error
import os
import re
import pandas as pd 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
from wordcloud import WordCloud, STOPWORDS
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import pymysql

class TweetObject():
	
	def __init__(self, host, database, user):
		self.password = 'ck1234'
		self.host = host
		self.database = database
		self.user = user
		


	def MySQLConnect(self,query):
		"""
		Connects to database and extracts
		raw tweets and any other columns we
		need
		Parameters:
		----------------
		arg1: string: SQL query
		Returns: pandas dataframr
		----------------
		"""

		try:
			con = pymysql.connect(host = self.host, database = self.database, \
				user = self.user, password = self.password,charset = 'utf8')

			if con.open:
				print("Successfully connected to database")
				cursor = con.cursor()
				query = query
				cursor.execute(query)

				result_data = cursor.fetchall()
				data = []
				for row in result_data:
					data.append({'date': row["created_at"], 'tweet': row["tweet"]})
				# store in dataframe
				#df = pd.DataFrame(data,columns = ['date', 'tweet'])
				#df = pd.read_sql_table(twitterdata,con,columns = ['date', 'tweet'])
				#print(df.head())



		except pymysql.Error as e:
			print(e)
		
		#cursor.close()
		con.close()

		
		return df
    
	def clean_tweets(self, df):
    	
    		"""
    		Takes raw tweets and cleans them
    		so we can carry out analysis
    		remove stopwords, punctuation,
    		lower case, html, emoticons.
    		This will be done using Regex
    		? means option so colou?r matches
    		both color and colour.
    		"""
    
    		# text preprocessing
    		stopword_list = stopwords.words('english')
    		#ps = PorterStemmer()
    		wordnet_lemmatizer = WordNetLemmatizer()
    		df["clean_tweets"] = None
    		df['len'] = None
    		for i in range(0,len(df['tweet'])):
    			# get rid of anything that isnt a letter
    
    			exclusion_list = ['[^a-zA-Z]','rt', 'http', 'co', 'RT']
    			exclusions = '|'.join(exclusion_list)
    			text = re.sub(exclusions, ' ' , df['tweet'][i])
    			text = text.lower()
    			words = text.split()
    			words = [wordnet_lemmatizer.lemmatize(word) for word in words if not word in stopword_list]
    			 # only use stem of word
    			#words = [ps.stem(word) for word in words]
    			df['clean_tweets'][i] = ' '.join(words)
    
    
    		# Create column with data length
    		df['len'] = np.array([len(tweet) for tweet in df["clean_tweets"]])
    			
    
    
    		return df    
    
	def sentiment(self, tweet):
		"""
		This function calculates sentiment
		on our cleaned tweets.
		Uses textblob to calculate polarity.
		Parameters:
		----------------
		arg1: takes in a tweet (row of dataframe)
		"""

		# need to improce
		analysis = TextBlob(tweet)
		if analysis.sentiment.polarity > 0:
			return 1
		elif analysis.sentiment.polarity == 0:
			return 0
		else:
			return -1




	def save_to_csv(self, df):
		"""
		Save cleaned data to a csv for further
		analysis.
		Parameters:
		----------------
		arg1: Pandas dataframe
		"""
		try:
			df.to_csv("clean_tweets.csv")
			print("\n")
			print("csv successfully saved. \n")

		
		except Error as e:
			print(e)
		



	def word_cloud(self, df):
		plt.subplots(figsize = (12,10))
		wordcloud = WordCloud(
				background_color = 'white',
				width = 1000,
				height = 800).generate(" ".join(df['clean_tweets']))
		plt.imshow(wordcloud)
		plt.axis('off')
		plt.show()

In [45]:
if __name__ == '__main__':

	t = TweetObject( host = 'localhost', database = 'twitterdb', user = 'root')

	data  = t.MySQLConnect("""SELECT created_at, tweet FROM twitterdata""")
	data = t.clean_tweets(data)
	data['Sentiment'] = np.array([t.sentiment(x) for x in data['clean_tweets']])
	t.word_cloud(data)
	t.save_to_csv(data)
	
	pos_tweets = [tweet for index, tweet in enumerate(data["clean_tweets"]) if data["Sentiment"][index] > 0]
	neg_tweets = [tweet for index, tweet in enumerate(data["clean_tweets"]) if data["Sentiment"][index] < 0]
	neu_tweets = [tweet for index, tweet in enumerate(data["clean_tweets"]) if data["Sentiment"][index] == 0]

	#Print results
	print("percentage of positive tweets: {}%".format(100*(len(pos_tweets)/len(data['clean_tweets']))))
	print("percentage of negative tweets: {}%".format(100*(len(neg_tweets)/len(data['clean_tweets']))))
	print("percentage of neutral tweets: {}%".format(100*(len(neu_tweets)/len(data['clean_tweets']))))